In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_train=pd.read_csv('/kaggle/input/mooc-data-xuetangx/Train.csv')
data_test=pd.read_csv('/kaggle/input/mooc-data-xuetangx/Test.csv')
user_profile=pd.read_csv('/kaggle/input/mooc-data-xuetangx/user_info (1).csv')

In [ ]:
df=pd.concat([data_train,data_test])
df

In [ ]:
user_profile=user_profile.rename(columns={"user_id": "username"})
user_profile

In [ ]:
df=df.merge(user_profile)
df['age']=2016-df['birth']
df.drop('birth',axis=1,inplace=True)

In [ ]:
user_enroll_num = df.groupby('username').count()[['course_id']]
user_enroll_num.columns = ['user_enroll_num']
user_enroll_num

In [ ]:
df=df.merge(user_enroll_num,left_on='username',right_index=True)
df

In [ ]:
df.isna().sum()/len(df)

In [ ]:
df.drop(['gender','education','age'],axis=1,inplace=True)

In [ ]:
# df.groupby('education')['age'].mean()

In [ ]:
# df['age'].median()

In [ ]:
# def impute_age(cols):
#     Age = cols[0]
#     education = cols[1]
#
#     if pd.isnull(Age):
#
#         if education == 'Associate':
#             return 32
#         elif education == "Bachelor's":
#             return 26
#         elif education in['Doctorate',"Master's"]:
#             return 31
#         elif education =='Primary':
#             return 30
#         elif education in ["High","Middle"]:
#             return 21
#
#         else:
#             return 25
#
#     else:
#         return Age

In [ ]:
from sklearn.model_selection import train_test_split


# Custom splitting function
def custom_train_test_split(data, test_size=0.2, random_state=None):
    unique_usernames = data['username'].unique()

    train_usernames, test_usernames = train_test_split(unique_usernames, test_size=test_size, random_state=random_state)

    train_data = data[(data['username'].isin(train_usernames))]
    test_data = data[(data['username'].isin(test_usernames))]

    return train_data, test_data

# Split the data
train_data, test_data = custom_train_test_split(df, test_size=0.2, random_state=42)



In [ ]:
train_data.drop(['username','course_id','session_id'],axis=1,inplace=True)
test_data.drop(['username','course_id','session_id'],axis=1,inplace=True)

In [ ]:
from catboost import CatBoostClassifier

x_train,y_train=train_data.drop('truth',axis=1),train_data['truth']
X_test,y_test=test_data.drop('truth',axis=1),test_data['truth']

In [ ]:
from catboost import CatBoostClassifier

x_train,y_train=train_data.drop('truth',axis=1),train_data['truth']
model = CatBoostClassifier(iterations=1000)



In [ ]:
model.fit(x_train,
          y_train,
          verbose=False)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve, auc
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier



# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate various evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"ROC AUC: {roc_auc:.2f}")
print("Confusion Matrix:")
print(conf_matrix)

# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

